# Scaling in Cantera
This notebook is where we figure out how to do linear scaling relationships, in Cantera

In [1]:
import sys
sys.executable = '/home/chao/anaconda3/envs/rmg_env/bin/python'
python_path = ['','/home/chao/cantera/build/python', '/home/chao/RMG-Py', '/home/chao', '/home/chao/anaconda3/envs/rmg_env/lib/python37.zip', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/lib-dynload', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/chemprop-0.0.1-py3.7.egg', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/descriptastorus-2.0.0.32-py3.7.egg']
sys.path.clear()
for path in python_path:
    sys.path.append(path)

In [1]:
import numpy as np
import scipy, scipy.optimize

%matplotlib inline
from matplotlib import pyplot as plt

import cantera as ct

import rmgpy
import rmgpy.data.base
import rmgpy.molecule
import rmgpy.quantity

In [2]:
def find_species_phase_index(species_name):
    """
    Return the phase object (gas or surface) and the index
    of the named species.
    """
    try:
        i = gas.species_index(species_name)
        return gas, i
    except ValueError:
        i = surf.species_index(species_name)
        return surf, i

def change_species_enthalpy(species_name, dH):
    """
    Find the species by name and change it's enthlapy by dH (in J/kmol)
    """
    phase, index = find_species_phase_index(species_name)

    species = phase.species(index)
    print(f"Initial H(298) = {species.thermo.h(298)/1e6:.1f} kJ/mol")
    dx = dH / ct.gas_constant  # 'dx' is in fact (delta H / R). Note that R in cantera is 8314.462 J/kmol
    assert isinstance(species.thermo, ct.NasaPoly2)
    # print(species.thermo.coeffs)
    perturbed_coeffs = species.thermo.coeffs.copy()
    perturbed_coeffs[6] += dx
    perturbed_coeffs[13] += dx
    
    species.thermo = ct.NasaPoly2(species.thermo.min_temp, species.thermo.max_temp, 
                            species.thermo.reference_pressure, perturbed_coeffs)
    #print(species.thermo.coeffs)
    phase.modify_species(index, species)
    print(f"Modified H(298) = {species.thermo.h(298)/1e6:.1f} kJ/mol")

# change_species_enthalpy('NH4NO3', 1e6) # 1 kJ/mol = 1e6 J/kmol
# change_species_enthalpy('NH4NO3', -1e6) # put it back for now

In [3]:
def correct_binding_energy(species, delta_atomic_adsoprtion_energies={}):
    """
    Changes the thermo of the provided species, by applying a linear scaling relation
    to correct the adsorption energy.

    :param species: The species to modify (an RMG Species object)
    :param delta_atomic_adsoprtion_energies: a dictionary of changes in atomic adsorption energies to apply.
        mapping for each element an RMG Quantity objects with .value_si giving a value in J/mol.
    :return: None
    """
    molecule = species.molecule[0]
    # only want/need to do one resonance structure
    surface_sites = []
    for atom in molecule.atoms:
        if atom.is_surface_site():
            surface_sites.append(atom)
    normalized_bonds = {'C': 0., 'O': 0., 'N': 0., 'H': 0.}
    max_bond_order = {'C': 4., 'O': 2., 'N': 3., 'H': 1.}
    for site in surface_sites:
        numbonds = len(site.bonds)
        if numbonds == 0:
            # vanDerWaals
            pass
        else:
            assert len(site.bonds) == 1, "Each surface site can only be bonded to 1 atom"
            bonded_atom = list(site.bonds.keys())[0]
            bond = site.bonds[bonded_atom]
            if bond.is_single():
                bond_order = 1.
            elif bond.is_double():
                bond_order = 2.
            elif bond.is_triple():
                bond_order = 3.
            elif bond.is_quadruple():
                bond_order = 4.
            else:
                raise NotImplementedError("Unsupported bond order {0} for binding energy "
                                          "correction.".format(bond.order))

            normalized_bonds[bonded_atom.symbol] += bond_order / max_bond_order[bonded_atom.symbol]


    # now edit the adsorptionThermo using LSR
    change_in_binding_energy = 0.0
    for element in delta_atomic_adsoprtion_energies.keys():
        change_in_binding_energy += delta_atomic_adsoprtion_energies[element].value_si * normalized_bonds[element]
    if change_in_binding_energy != 0.0:
        print(f"Applying LSR correction to {species.label}:")
        change_species_enthalpy(species.label, change_in_binding_energy*1000)

In [4]:
# Create Gas Solution and Surface Interface from cti file
cti_file = "./chem_annotated_Pt_large.cti" # path to cti file
gas = ct.Solution(cti_file)
surf = ct.Interface(cti_file,'surface1', [gas])
species_dict = rmgpy.data.base.Database().get_species('species_dictionary_Pt_large.txt',resonance=False)
#species_dict = rmgpy.data.base.Database().get_species('../RMG-model/chemkin/species_dictionary.txt',resonance=False)

In [5]:
delta_atomic_adsoprtion_energies = {
    "C" : rmgpy.quantity.Energy(-2,'kcal/mol'),
    "O" : rmgpy.quantity.Energy(3,'kcal/mol'),
    "N" : rmgpy.quantity.Energy(-5,'kcal/mol'),
}

In [6]:
for species in surf.species():
    rmg_spcs = species_dict[species.name]
    correct_binding_energy(rmg_spcs, delta_atomic_adsoprtion_energies)
    

Applying LSR correction to HO_Pd(30):
Initial H(298) = -155.9 kJ/mol
Modified H(298) = -149.6 kJ/mol
Applying LSR correction to H2NOX(31):
Initial H(298) = -145.0 kJ/mol
Modified H(298) = -138.7 kJ/mol
Applying LSR correction to N_Pt(35):
Initial H(298) = 40.1 kJ/mol
Modified H(298) = 19.2 kJ/mol
Applying LSR correction to O_Pt(36):
Initial H(298) = -140.5 kJ/mol
Modified H(298) = -127.9 kJ/mol
Applying LSR correction to NO_Pt(37):
Initial H(298) = -107.6 kJ/mol
Modified H(298) = -114.6 kJ/mol
Applying LSR correction to CH3X(39):
Initial H(298) = -45.8 kJ/mol
Modified H(298) = -47.9 kJ/mol
Applying LSR correction to OCX(41):
Initial H(298) = -281.4 kJ/mol
Modified H(298) = -285.6 kJ/mol
Applying LSR correction to CX(42):
Initial H(298) = 53.6 kJ/mol
Modified H(298) = 45.3 kJ/mol
Applying LSR correction to CH2X(43):
Initial H(298) = 0.7 kJ/mol
Modified H(298) = -3.5 kJ/mol
Applying LSR correction to CHX(44):
Initial H(298) = -25.4 kJ/mol
Modified H(298) = -31.7 kJ/mol
Applying LSR corre

In [7]:
species_dict

OrderedDict([('SX(26)',
              Species(label="SX(26)", molecule=[Molecule(smiles="[O-][N+](=O)[O-].[Pt].[NH4+]")], molecular_weight=(80.0434,'amu'))),
             ('N2O_X(27)',
              Species(label="N2O_X(27)", molecule=[Molecule(smiles="[N-]=[N+]=O.[Pt]")], molecular_weight=(44.0129,'amu'))),
             ('H2O_X(28)',
              Species(label="H2O_X(28)", molecule=[Molecule(smiles="O.[Pt]")], molecular_weight=(18.0153,'amu'))),
             ('HAN_X(29)',
              Species(label="HAN_X(29)", molecule=[Molecule(smiles="[O-][N+](=O)[O-].O[NH3+].[Pt]")], molecular_weight=(96.0428,'amu'))),
             ('HO_Pd(30)',
              Species(label="HO_Pd(30)", molecule=[Molecule(smiles="O[Pt]")], molecular_weight=(17.0073,'amu'))),
             ('H2NOX(31)',
              Species(label="H2NOX(31)", molecule=[Molecule(smiles="NO[Pt]")], molecular_weight=(32.022,'amu'))),
             ('HONO(32)',
              Species(label="HONO(32)", molecule=[Molecule(smiles="ON=O")],